# 梯度和变分优化

## 概述

TyxonQ 旨在使参数化量子门的优化变得简单、快速和方便。 在本说明中，我们回顾了如何获得电路梯度和运行变分优化。

## 设置

In [ ]:
import numpy as np
import scipy.optimize as optimize
import torch
import tyxonq as tq

K = tq.set_backend("pytorch")

## PQC(Parameterized Quantum Circuit)

考虑一个作用于 $n$ 个量子比特的变分电路，由 $k$ 层组成，其中每一层包含相邻量子比特之间的参数化 $e^{i\theta X\otimes X}$ 门，
然后是一系列参数化的单量子比特 $Z$ 和 $X$ 旋转。
我们现在展示如何在 TyxonQ 中实现此类电路，以及如何使用机器学习后端之一轻松高效地计算损失函数和梯度。

一般$n,k$的电路和参数集可以定义如下：:

In [ ]:
def qcircuit(n, k, params):
    c = tq.Circuit(n)
    for j in range(k):
        for i in range(n - 1):
            c.exp1(
                i, i + 1, theta=params[j * (3 * n - 1) + i], unitary=tq.gates._xx_matrix
            )
        for i in range(n):
            c.rz(i, theta=params[j * (3 * n - 1) + n - 1 + i])
            c.rx(i, theta=params[j * (3 * n - 1) + 2 * n - 1 + i])
    return c

举个例子，我们取 $n=3, k=2$ ，设置 PyTorch 作为我们的后端，定义一个能量损失函数来最小化
$$E = \langle X_0 X_1\rangle_\theta + \langle X_1 X_2\rangle_\theta.$$

In [4]:
n = 3
k = 2


def energy(params):
    c = qcircuit(n, k, params)
    e = c.expectation_ps(x=[0, 1]) + c.expectation_ps(x=[1, 2])
    return K.real(e)

## 梯度和即时编译

使用 ML(Machine Learning) 后端对自动微分的支持，我们现在可以快速计算能量和能量相对于参数的梯度。

In [5]:
energy_val_grad = K.value_and_grad(energy)

这将创建一个函数，给定一组参数作为输入，返回能量和能量梯度。如果只需要梯度，则可以通过 ``K.grad(energy)`` 计算。
虽然我们可以直接在一组参数上运行上述代码，但如果要对能量进行多次评估，则可以通过使用该函数的即时编译版本来节省大量时间。

In [6]:
energy_val_grad_jit = K.jit(energy_val_grad)

使用 ``K.jit``，能量和梯度的初始评估可能需要更长的时间，但随后的评估将明显快于非 jitted 代码。
我们建议始终使用 ``jit``，只要函数是 ``张量输入，张量输出`` 的形式，我们已经努力使电路模拟器的各个方面都与 JIT 兼容。

## 通过 ML(Machine Learning)  后端进行优化

有了可用的能量函数和梯度，参数的优化就很简单了。下面是一个如何通过随机梯度下降来做到这一点的例子。

In [ ]:
learning_rate = 2e-2
# warning pytorch might be unable to do this exactly
params = torch.nn.Parameter(torch.randn(k * (3 * n - 1)))
opt = torch.optim.SGD([params], lr=learning_rate)


def grad_descent(params, i):
    val, grad = energy_val_grad_jit(params)
    params.grad = grad
    opt.step()
    if i % 10 == 0:
        print(f"i={i}, energy={val}")
    return params


for i in range(100):
    params = grad_descent(params, i)

i=0, energy=-0.670495331287384
i=10, energy=-1.2501306533813477
i=20, energy=-1.4198601245880127
i=30, energy=-1.5485260486602783
i=40, energy=-1.6695808172225952
i=50, energy=-1.7761402130126953
i=60, energy=-1.8588359355926514
i=70, energy=-1.915252923965454
i=80, energy=-1.9499529600143433
i=90, energy=-1.9699469804763794


## 通过 Scipy 界面进行优化

使用机器学习后端进行优化的另一种方法是使用 SciPy。
这可以通过 ``scipy_interface`` API 调用来完成，并允许使用基于梯度（例如 BFGS）和非基于梯度（例如 COBYLA）的优化器，这在 ML(Machine Learning)  后端是不可用的。

In [ ]:
f_scipy = tq.interfaces.scipy_interface(energy, shape=[k * (3 * n - 1)], jit=True)
params = torch.randn(k * (3 * n - 1)).numpy()
r = optimize.minimize(f_scipy, params, method="L-BFGS-B", jac=True)
r

  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: -2.0000014305114746
        x: [-7.126e-01 -2.315e+00 ...  1.007e+00  5.574e-01]
      nit: 16
      jac: [ 7.153e-05  1.353e-04 ...  0.000e+00  0.000e+00]
     nfev: 50
     njev: 50
 hess_inv: <16x16 LbfgsInvHessProduct with dtype=float64>

上面的第一行指定了要提供给要最小化的函数的参数的形状，这里是能量函数。
``jit=True`` 参数会自动处理能量函数的即时编译。 通过将 ``gradient=False`` 参数提供给``scipy_interface``，同样可以有效地执行无梯度优化。

In [ ]:
f_scipy = tq.interfaces.scipy_interface(
    energy, shape=[k * (3 * n - 1)], jit=True, gradient=False
)
params = torch.randn(k * (3 * n - 1)).numpy()
r = optimize.minimize(f_scipy, params, method="COBYLA")
r

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.9999998807907104
       x: [ 7.857e-01  7.854e-01 ...  2.270e-01  6.234e-01]
    nfev: 278
   maxcv: 0.0